# Annulus
* Let `P` and `Q` be two point cloud sampled from two distinct distributions on an annulus.
* Let `Q` and `R` be two point clouds sampled from the same distributions on an annulus.

## 1. Comparison of `P` and `Q`
* This notebook illustrates the problem of constructing persistence modules $$VR(P) \rightarrow VR(P \cup Q) \leftarrow VR(Q)$$
* This notebook also implements the similarity-centric analogous bars method.

## 2. Comparison of `Q` and `R`
* This notebook contains a comparison to existing methods (induced matching, cycle registration) and application of the analogous bars method.

## 3. Comparison of `P` and `R`

#### Outline
1. Load points
2. Application of induced matching and cycle registration
3. Application of similarity-centric analogous bars method.

In [1]:
include("../../../extension_method.jl")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-6452897574250202482\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-16567600875401435217\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-6452897574250202482\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /opt/julia/packages/ORCA/U5XaN/src/ORCA.jl:8


Main.ext

In [2]:
using .ext
using Distances
using Distributions
using Eirene
using Plots
using JLD
using Measures

# 0. Load points

Note: The points were generated using the following code

In [3]:
function sample_from_annulus(;mean = 1, variance = 0.3, n = 20)
    samples = zeros(n, 2)
    theta = rand(Uniform(0, 2*pi), n)
    distance = rand(Normal(mean, variance), n)
    
    for i=1:n
        samples[i, 1] = distance[i] * cos(theta[i]) 
        samples[i, 2] = distance[i] * sin(theta[i])
    end
    return samples
end

sample_from_annulus (generic function with 1 method)

In [76]:
n_small = 30
n_large = 20

P = sample_from_annulus(mean = 0.5, variance = 0.1, n = n_small)
Q = sample_from_annulus(mean = 1, variance = 0.3, n = n_large)
R = sample_from_annulus(mean = 1, variance = 0.3, n = n_large);

#save("three_point_clouds.jld", "P", P, "Q", Q, "R", R)

In [3]:
# load points
saved = load("three_point_clouds.jld")
P = saved["P"]
Q = saved["Q"]
R = saved["R"];

┌ Warning: JLD incorrectly extends FileIO functions (see FileIO documentation)
└ @ FileIO /opt/julia/packages/FileIO/JA3Vl/src/loadsave.jl:215


In [4]:
plot_P_Q(P, Q)

<!DOCTYPE html>
 
 
 Plots.jl

In [5]:
plot_P_Q(Q, R, P_label = "Q", Q_label = "R")

<!DOCTYPE html>
 
 
 Plots.jl

# 1. Comparison of $P$ and $Q$ 

## 1(a) Plot barcodes of $P$, $Q$, and $P \cup Q$

In [101]:
# compute distances among points
P_all = vcat(P, Q)
D_PQ = pairwise(Euclidean(), transpose(P_all), transpose(P_all), dims=2)

# Define submatrices 
n_P = size(P, 1)
D_P = D_PQ[1:n_P, 1:n_P]
D_Q = D_PQ[n_P+1:end, n_P+1:end]
D_P_Q = D_PQ[1:n_P, n_P+1:end]
    # rows (landmarks): P
    # columns (witness) : Q
D_Q_P = D_PQ[n_P+1:end, 1:n_P];
    # rows (landmarks): Q
    # columns (witness) : P

In [102]:
# run persistence
C_P = eirene(D_P, record = "all")
C_PQ = eirene(D_PQ, record = "all")
C_Q = eirene(D_Q, record = "all");

In [103]:
# plot all barcodes in dim 1
barcode_P = barcode(C_P, dim = 1)
barcode_PQ = barcode(C_PQ, dim = 1)
barcode_Q = barcode(C_Q, dim = 1)

l = grid(3,1)
p1 = plot_barcode(barcode_P, lw = 3, title = "Barcode(VR(P))", titlefontsize = 12)
p2 = plot_barcode(barcode_PQ, lw = 3, title = "Barcode(VR(P U Q))", titlefontsize = 12)
p3 = plot_barcode(barcode_Q, lw = 3, title = "Barcode(VR(Q))", titlefontsize = 12)
plot(p1, p2, p3, layout = l, size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl

The features of P and Q exist at very different scales. Therefore, any attempt to matching between `barcode(VR(Q))` and `barcode(VR(P U Q))` will be trivial.

## 1(b) Implement the similarity-centric analogous bars method

In [93]:
# Compute Vietoris-Rips persistence 
dim = 1
VR_P = eirene(D_P, record = "all", maxdim = dim)
VR_Q = eirene(D_Q, record = "all", maxdim = dim )

# Compute Witness filtration
W_P = compute_Witness_persistence(D_P_Q, maxdim = dim)
W_Q = compute_Witness_persistence(D_Q_P, maxdim = dim);

In [94]:
# plot all four barcodes
barcode_P = barcode(VR_P, dim = 1)
barcode_W_PQ = barcode(W_P["eirene_output"], dim = 1)
barcode_W_QP = barcode(W_Q["eirene_output"], dim = 1)
barcode_Q = barcode(VR_Q, dim = 1)

l = grid(4,1)
p1 = plot_barcode(barcode_P, lw = 3, title = "Barcode(VR(P))", titlefontsize = 12)
p2 = plot_barcode(barcode_W_PQ, lw = 3, title = "Barcode(W(P,Q))", titlefontsize = 12)
p3 = plot_barcode(barcode_W_QP, lw = 3, title = "Barcode(W(Q,P))", titlefontsize = 12)
p4 = plot_barcode(barcode_Q, lw = 2, title = "Barcode(VR(Q))", titlefontsize = 12)
plot(p1, p2, p3, p4, layout = l, size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl

In [95]:
# select Witness bar
W_PQ_bar = 1

# run extension
extension_P, extension_Q = run_similarity_analogous(VR_P = VR_P, 
                                                    D_P = D_P, 
                                                    VR_Q = VR_Q, 
                                                    D_Q = D_Q, 
                                                    W_PQ = W_P, 
                                                    W_PQ_bar = W_PQ_bar, 
                                                    dim = dim);

In [96]:
plot_analogous_bars(extension_P, extension_Q, lw_VR_P = 8)

<!DOCTYPE html>
 
 
 Plots.jl

# 2. Comparison of `Q` and `R`

## 2(a). Plot barcodes on $Q$, $R$, and $Q \cup R$

In [13]:
# compute distances among points
same_dist = vcat(Q, R)
D_QR = pairwise(Euclidean(), transpose(same_dist), transpose(same_dist), dims=2)

# Define submatrices
n_Q = size(Q, 1)
D_Q = D_QR[1:n_Q, 1:n_Q]
D_R = D_QR[n_Q+1:end, n_Q+1:end]
D_Q_R = D_QR[1:n_Q, n_Q+1:end]
    # rows (landmarks): Q
    # columns (witness) : R
D_R_Q = D_QR[n_Q+1:end, 1:n_Q];
    # rows (landmarks): R
    # columns (witness) : Q

In [14]:
# run persistence
C_Q = eirene(D_Q, record = "all")
C_QR = eirene(D_QR, record = "all")
C_R = eirene(D_R, record = "all");

In [15]:
# plot all barcodes in dim 1
barcode_Q = barcode(C_Q, dim = 1)
barcode_QR = barcode(C_QR, dim = 1)
barcode_R = barcode(C_R, dim = 1)

l = grid(3,1)
p1 = plot_barcode(barcode_Q, lw = 3, title = "Barcode(VR(Q))", titlefontsize = 12)
p2 = plot_barcode(barcode_QR, lw = 3, title = "Barcode(VR(Q U R))", titlefontsize = 12)
p3 = plot_barcode(barcode_R, lw = 3, title = "Barcode(VR(R))", titlefontsize = 12)
plot(p1, p2, p3, layout = l, size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl

## 2(b) Induced matching / cycle registration  
* We have $VR(Q) \rightarrow VR(Q \cup R) \leftarrow VR(R)$ 
* To see the results of induced matching / cycle registration on this dataset, we need to find the induced matching between $VR(Q) \rightarrow VR(Q \cup R)$, and $VR(R) \rightarrow VR(Q \cup R)$ 
* To find induced matching for $f: VR(Q) \rightarrow VR(Q \cup R)$, we need to compute the image persistence barcode $im(f)$. I'm not sure if there is a software for this, so for now, we'll just use a 'hack':
    * For an interval $I$ in $VR(Q)$, find its cycle rep. Find the death time of this cycle in $VR(Q \cup R)$. If there exists an interval $I_* \in barcode(VR(Q \cup R))$, then induced matching will map $I$ to $I_*$. 
    * Similarly for induced matching via $g: VR(R) \to VR(Q \cup R)$.

### 2(b)(i) Induced matching $ f: VR(Q) \to VR(Q \cup R) $
* From the induced matching paper (2015) Theorem 6.1, if an interval $[b,d] \in barcode(VR(Q))$ matches to $[b', d'] \in barcode(VR(Q \cup R))$, they must satisfy $b' \leq b < d' \leq d$.
* The unique interval (interval 1) of $barcode(VR(Q))$ has a birth time of $b = 1.05$. In $barcode(VR(Q \cup R))$, the latest death time is at $0.93$. So all $d' < b$. Interval 1 of $barcode(VR(Q))$ thus doesn't match to any intervals in $barcode(VR(Q \cup R))$. 

### 2(b)(ii) Induced matching $g: VR(R) \to VR(Q \cup R) $
* To compute the induced matching between $barcode(VR(R))$ and $barcode(VR(Q \cup R))$, we need to find the barcode of the image persistence $im (g)$. 
* I'm not sure if there's any software for computing the barcode of the image persistence. So I just checked for the death time of interval 5 in $VR(Q \cup R)$. 
* The following code computes the death time of cycle 5 from $VR(R)$ in $VR(Q \cup R)$

In [33]:
S = classrep(C_R, class = 5, dim = 1)

# cyclerep as [simplex1, simplex2, ..., simplexk], where each simplex is a list of vertices
chain_vertices = []
for j=1:size(S)[2]
    append!(chain_vertices, [sort(S[:,j])])
end
n_Q = size(Q, 1)
chain_vertices = [item + [n_Q, n_Q] for item in chain_vertices]
chain = ext.chain_to_index(chain_vertices, C_union)
Eirene.deathtime(C_union, chain = chain, dim = 1)

0.9295147917125612

From looking at the death time, we know that interval 5 of $VR(R)$ matches to interval 5 of $VR(Q \cup R)$.

## 2(c). Implement the similarity-centric analogous bars method


In [57]:
# Compute Vietoris-Rips persistence on two regions
dim = 1
VR_Q = eirene(D_Q, record = "all", maxdim = dim)
VR_R = eirene(D_R, record = "all", maxdim = dim )

# compute Witness persistence
W_QR = compute_Witness_persistence(D_Q_R, maxdim = dim)
W_RQ = compute_Witness_persistence(D_R_Q, maxdim = dim)

barcode_W_QR = barcode(W_QR["eirene_output"], dim = 1)
barcode_W_RQ = barcode(W_RQ["eirene_output"], dim = 1);

In [42]:
# plot all four barcodes
l = grid(4,1)
p1 = plot_barcode(barcode(VR_Q, dim = 1), lw = 3, title = "Barcode(VR(Q))", titlefontsize = 12)
p2 = plot_barcode(barcode(W_QR["eirene_output"], dim = 1) , lw = 3, title = "Barcode(W(Q,R))", titlefontsize = 12)
p3 = plot_barcode(barcode(W_RQ["eirene_output"], dim = 1) , lw = 3, title = "Barcode(W(R,Q))", titlefontsize = 12)
p4 = plot_barcode(barcode(VR_R, dim = 1), lw = 2, title = "Barcode(VR(R))", titlefontsize = 12)
plot(p1, p2, p3, p4, layout = l, size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl

In [43]:
# select interval
W_QR_bar = 3

# run similarity-centric analogous bars method
extension_Q, extension_R = run_similarity_analogous(VR_P = VR_Q, 
                                                    D_P = D_Q, 
                                                    VR_Q = VR_R, 
                                                    D_Q = D_R, 
                                                    W_PQ = W_QR, 
                                                    W_PQ_bar = W_QR_bar, 
                                                    dim = dim);

In [44]:
plot_analogous_bars(extension_P, extension_Q)

<!DOCTYPE html>
 
 
 Plots.jl

In [104]:
# save all relevant barcodes
save("annulus_barcodes.jld", 
    "barcode_P", barcode_P,
    "barcode_Q", barcode_Q,
    "barcode_R", barcode_R,
    "barcode_PQ", barcode_PQ,
    "barcode_QR", barcode_QR,
    "barcode_W_QR", barcode_W_QR,
    "barcode_W_RQ", barcode_W_RQ,
    "int_QR", [5],
    "int_R", [5],
    "bar_Q", [1],
    "bar_W_QR", [3],
    "bar_W_RQ", [3],
    "bar_R", [5]
    )

┌ Warning: JLD incorrectly extends FileIO functions (see FileIO documentation)
└ @ FileIO /opt/julia/packages/FileIO/JA3Vl/src/loadsave.jl:215


# 3. Comparison of $P$ and $R$

## 3(a) Plot barcodes of $P$, $R$, and $P \cup R$

In [6]:
# compute distances among points
PR_all = vcat(P, R)
D_PR = pairwise(Euclidean(), transpose(PR_all), transpose(PR_all), dims=2)

# Define submatrices 
n_P = size(P, 1)
D_P = D_PR[1:n_P, 1:n_P]
D_R = D_PR[n_P+1:end, n_P+1:end]
D_P_R = D_PR[1:n_P, n_P+1:end]
    # rows (landmarks): P
    # columns (witness) : R
D_R_P = D_PR[n_P+1:end, 1:n_P];
    # rows (landmarks): R
    # columns (witness) : P

In [9]:
# run persistence
C_P = eirene(D_P, record = "all")
C_PR = eirene(D_PR, record = "all")
C_R = eirene(D_R, record = "all");

In [12]:
# plot all barcodes in dim 1
barcode_P = barcode(C_P, dim = 1)
barcode_PR = barcode(C_PR, dim = 1)
barcode_R = barcode(C_R, dim = 1)

l = grid(3,1)
p1 = plot_barcode(barcode_P, lw = 3, title = "Barcode(VR(P))", titlefontsize = 12)
p2 = plot_barcode(barcode_PR, lw = 3, title = "Barcode(VR(P U R))", titlefontsize = 12)
p3 = plot_barcode(barcode_R, lw = 3, title = "Barcode(VR(R))", titlefontsize = 12)
plot(p1, p2, p3, layout = l, size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl

### 3(b)(i) Induced matching $f: VR(P) \to VR(P \cup R) $
* To compute the induced matching between $barcode(VR(P))$ and $barcode(VR(P \cup R))$, we need to find the barcode of the image persistence $im (f)$. 
* I'm not sure if there's any software for computing the barcode of the image persistence. So I just checked for the death time of interval 1 (from barcode(VR(P)) ) in $VR(P \cup R)$. 
* The following code computes the death time of cycle 1 from $VR(P)$ in $VR(P \cup R)$

In [16]:
S = classrep(C_P, class = 1, dim = 1)

# cyclerep as [simplex1, simplex2, ..., simplexk], where each simplex is a list of vertices
chain_vertices = []
for j=1:size(S)[2]
    append!(chain_vertices, [sort(S[:,j])])
end
#n_Q = size(Q, 1)
#chain_vertices = [item + [n_Q, n_Q] for item in chain_vertices]
chain = ext.chain_to_index(chain_vertices, C_PR)
Eirene.deathtime(C_PR, chain = chain, dim = 1)

0.8092713525599627

From looking at the death time, we know that interval 1 of $VR(P)$ matches to interval 6 of $VR(P \cup R)$.

### 3(b)(ii) Induced matching $g: VR(R) \to VR(P \cup R) $
* To compute the induced matching between $barcode(VR(R))$ and $barcode(VR(P \cup R))$, we need to find the barcode of the image persistence $im (g)$. 
* I'm not sure if there's any software for computing the barcode of the image persistence. So I just checked for the death time of interval 5 (from barcode(VR(R)) ) in $VR(P \cup R)$. 
* The following code computes the death time of cycle 5 from $VR(R)$ in $VR(P \cup R)$

In [19]:
S = classrep(C_R, class = 5, dim = 1)

# cyclerep as [simplex1, simplex2, ..., simplexk], where each simplex is a list of vertices
chain_vertices = []
for j=1:size(S)[2]
    append!(chain_vertices, [sort(S[:,j])])
end

n_P = size(P, 1)
chain_vertices = [item + [n_P, n_P] for item in chain_vertices]
chain = ext.chain_to_index(chain_vertices, C_PR)
Eirene.deathtime(C_PR, chain = chain, dim = 1)

0.8092713525599627

From looking at the death time, we know that interval 5 of $VR(R)$ matches interval 6 of $VR(P \cup R)$.

In [20]:
l = grid(3,1)
p1 = plot_barcode(barcode_P, lw = 3, selected_bars = [1], title = "Barcode(VR(P))", titlefontsize = 12)
p2 = plot_barcode(barcode_PR, lw = 3, selected_bars = [6], title = "Barcode(VR(P U R))", titlefontsize = 12)
p3 = plot_barcode(barcode_R, lw = 3, selected_bars = [5], title = "Barcode(VR(R))", titlefontsize = 12)
plot(p1, p2, p3, layout = l, size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl